In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:07
🔁 Restarting kernel...


In [ ]:
!mamba install -c conda-forge rdkit


Looking for: ['rdkit']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.10.*
  - python 3.10.*
  - python_abi 3.10.* *cp310*
  - cudatoolkit *.*.*


Transaction

  Prefix: /usr/local

  All requested packages already installed



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import Draw

from IPython.display import display

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Data

/content/drive/.shortcut-targets-by-id/1lU6QjJaBcElIvfuufmZL2sO12RkRuYUm/Data


In [ ]:
ls

adata_excluded_ids.csv  condacolab_install.log  leash-BELKA.zip         multiome_var_meta.csv
adata_obs_meta.csv      de_train.parquet        multiome_obs_meta.csv   sample_submission.csv
adata_train.parquet     id_map.csv              multiome_train.parquet


## Importar bases


In [ ]:
adata_excluded = pd.read_csv('adata_excluded_ids.csv')

In [ ]:
adata_obs = pd.read_csv('adata_obs_meta.csv')

In [ ]:
adata_train = pd.read_parquet('adata_train.parquet')

In [ ]:
de_train = pd.read_parquet('de_train.parquet')

In [15]:
id_map = pd.read_csv('id_map.csv')
id_map.head()

,id,cell_type,sm_name
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
1,1,B cells,ABT-199 (GDC-0199)
2,2,B cells,ABT737
3,3,B cells,AMD-070 (hydrochloride)
4,4,B cells,AT 7867


In [ ]:
multiome_obs = pd.read_csv('multiome_obs_meta.csv')

In [ ]:
multiome_train = pd.read_parquet('multiome_train.parquet')

In [ ]:
multiome_var = pd.read_csv('multiome_var_meta.csv')

In [ ]:
sample_submission = pd.read_csv('sample_submission.csv')